# IBM Data Science Professional Specialization - Coursera

## Applied Data Science - Capstone Project - The Battle of Neighborhoods (Week 2) 

This notebook is part of the capstone project for the final course of the IBM Data Science professional Specialization program on Coursera.
This goal of this project is to apply the tools taught in the course to answer a self-stated research question using location data retrieved from Foursquare's API.

## Data Collection and Cleaning

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns

from ipynb.fs.full.Modules import getLatLng, getNearbyVenues 

In C:\Users\Simon\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Simon\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Simon\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Simon\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Simon\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
df = pd.read_csv('ames_train.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df.describe(include = 'all')

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1460.000000,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460.000000,1460.000000,1460.000000,1460.000000,1460,1460,1460,1460,1452,1452.000000,1460,1460,1460,1423,1423,1422,1423,1460.000000,1422,1460.000000,1460.000000,1460.000000,1460,1460,1460,1459,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460,1460.000000,1460,1460.000000,770,1379,1379.000000,1379,1460.000000,1460.000000,1379,1379,1460,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,5,3,25,9,8,5,8,NaN,NaN,NaN,NaN,6,8,15,16,4,NaN,4,5,6,4,4,4,6,NaN,6,NaN,NaN,NaN,6,5,2,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,7,NaN,5,6,NaN,3,NaN,NaN,5,5,3,NaN,NaN,NaN,NaN,NaN,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,NaN,NaN,NaN,NaN,Gable,CompShg,VinylSd,VinylSd,None,NaN,TA,TA,PConc,TA,TA,No,Unf,NaN,Unf,NaN,NaN,NaN,GasA,Ex,Y,SBrkr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TA,NaN,Typ,NaN,Gd,Attchd,NaN,Unf,NaN,NaN,TA,TA,Y,NaN,NaN,NaN,NaN,NaN,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,1151,NaN,NaN,1454,50,925,1311,1459,1052,1382,225,1260,1445,1220,726,NaN,NaN,NaN,NaN,1141,1434,515,504,864,NaN,906,1282,647,649,1311,953,430,NaN,1256,NaN,NaN,NaN,1428,741,1365,1334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,735,NaN,1360,NaN,380,870,NaN,605,NaN,NaN,1311,1326,1340,NaN,NaN,NaN,NaN,NaN,NaN,3,157,49,NaN,NaN,NaN,1267,1198,NaN
mean,730.500000,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.099315,5.575342,1971.267808,1984.865753,NaN,NaN,NaN,NaN,NaN,103.685262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443.639726,NaN,46.549315,567.240411,1057.429452,NaN,NaN,NaN,NaN,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,NaN,6.517808,NaN,0.613014,NaN,NaN,1978.506164,NaN,1.767123,472.980137,NaN,NaN,NaN,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN,180921.195890
std,421.610009,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.382997,1.112799,30.202904,20.645407,NaN,NaN,NaN,NaN,NaN,181.066207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,456.098091,NaN,161.319273,441.866955,438.705324,NaN,NaN,NaN,NaN,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,NaN,1.625393,NaN,0.644666,NaN,NaN,24.689725,NaN,0.747315,213.804841,NaN,NaN,NaN,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1872.000000,1950.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2.000000,NaN,0.000000,NaN,NaN,1900.000000,NaN

Quick notes and reduction:

Since 1stFlrSF + 2ndFlrSF == GrLivArea, we drop 1stFlrSF and 2ndFlrSF

Also MSSubClass seem to be a collection of variables (YearBuilt, HouseStyle, BldgTyp), so we drop it

In [5]:
df[['1stFlrSF', '2ndFlrSF']].sum(axis = 1).corr(df['GrLivArea']).round(2)

df.drop(['1stFlrSF', '2ndFlrSF', 'MSSubClass'], axis = 1, inplace = True)

In [6]:
# Fill NA's
null = df.isnull().sum().sort_values(ascending=False)
null = null.drop(null[null==0].index)
null / len(df)

PoolQC          0.995205
MiscFeature     0.963014
Alley           0.937671
Fence           0.807534
FireplaceQu     0.472603
LotFrontage     0.177397
GarageType      0.055479
GarageCond      0.055479
GarageYrBlt     0.055479
GarageFinish    0.055479
GarageQual      0.055479
BsmtExposure    0.026027
BsmtFinType2    0.026027
BsmtCond        0.025342
BsmtQual        0.025342
BsmtFinType1    0.025342
MasVnrArea      0.005479
MasVnrType      0.005479
Electrical      0.000685
dtype: float64

### Dealing with missing values

###### Pool
Since most houses doesn't have pools, PoolQC is mainly NA. We convert it to a binary variable, but must be aware of the limited sample of houses with pools (7 observations). 

###### LotFrontage
LotFrontage is 'Linear feet of street connected to property', for missing values, we will insert the mean value

###### MiscFeature, Alley, Fence, Fireplace, Garage and Bsmt
MiscFeature, Alley, Fence, Fireplace all have the same issue. Few houses have these features and we therefore substiture with 'NA' to have a new categorical feature capturing the zero case. Same for missing garage and bsmt features. For the LotFrontage we assign the mean value.

GarageYrBlt has missing values, we could drop this variable as it is highly correlated (0.83) with YearBuilt.   

###### Electrical
Since electrical is highly skewed towards all houses having SBrkr, we will assign this value to the single missing one.

###### MasVnrType and MasVnrArea
Missing values replaced with 'NA' and 0

In [7]:
# Pool to binary and MiscFeature, Alley, Fence, FireplaceQu to NA
df.loc[~df['PoolQC'].isnull(),'PoolQC'] = 1
df['PoolQC'].fillna(0, inplace = True)
df['MiscFeature'].fillna('None', inplace = True)
df['Alley'].fillna('None', inplace = True)
df['Fence'].fillna('None', inplace = True)
df['FireplaceQu'].fillna('None', inplace = True)

# LotFrontage
df.loc[df['LotFrontage'].isna(), 'LotFrontage'] = df['LotFrontage'].mean()

# Garage
df['GarageCond'].fillna('None', inplace = True)
df['GarageType'].fillna('None', inplace = True)
df['GarageQual'].fillna('None', inplace = True)
df['GarageFinish'].fillna('None', inplace = True)

# Basement
df['BsmtCond'].fillna('None', inplace = True)
df['BsmtQual'].fillna('None', inplace = True)
df['BsmtFinType1'].fillna('None', inplace = True)
df['BsmtFinType2'].fillna('None', inplace = True)
df['BsmtExposure'].fillna('None', inplace = True)

# Electrical
df['Electrical'].fillna('SBrkr', inplace = True)

# MasVnrType,MasVnrArea
df['MasVnrType'].fillna('None', inplace = True)
df['MasVnrArea'].fillna(0, inplace = True)

In [8]:
df.drop('GarageYrBlt', axis = 1, inplace = True)

In [9]:
# Still missing values?
null = df.isnull().sum().sort_values(ascending=False)
null = null.drop(null[null==0].index)
null / len(df)

Series([], dtype: float64)

### Foursquare API

In [10]:
# Dict for refering between ames data and foursquare
nbh = {'Blmngtn': 'Bloomington Heights',
       'Blueste': 'Bluestem',
       'BrDale' :'Briardale',
       'BrkSide':'Brookside',
       'ClearCr':'Clear Creek',
       'CollgCr':'College Creek',
       'Crawfor':'Crawford',
       'Edwards':'Edwards',
       'Gilbert':'Gilbert',
       'IDOTRR':'Iowa DOT and Rail Road',
       'MeadowV':'Meadow Village',
       'Mitchel':'Mitchell',
       'Names':'North Ames',
       'NoRidge':'Northridge',
       'NPkVill':'Northpark Villa',
       'NridgHt':'Northridge Heights',
       'NWAmes':'Northwest Ames',
       'OldTown':'Old Town',
       'SWISU':'South & West of Iowa State University',
       'Sawyer':'Sawyer',
       'SawyerW':'Sawyer West',
       'Somerst':'Somerset',
       'StoneBr':'Stone Brook',
       'Timber':'Timberland',
       'Veenker': 'Veenker'}

In [11]:
# Define function to get latitude and longitude for each neighborhood
API_KEY = 'b6b42744d9454990a83302a20001bc38'

In [12]:
#Get latitude and longitudes
nbh_list = list(nbh.values())
nbh_lat, nbh_lng = getLatLng(nbh_list, API_KEY)

# Convert to dataframe
nbh_dict = {'Latitude': nbh_lat, 'Longitude': nbh_lng}
nbh_df = pd.DataFrame(nbh_dict)
nbh_df['Neighborhood'] = nbh_list
nbh_df.head()

# Merge dfs
df['Neighborhood'] = df['Neighborhood'].map(nbh)
df = df.merge(nbh_df, on = 'Neighborhood')
df.head()

,Id,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Latitude,Longitude
0,1,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,College Creek,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,None,None,0,2,2008,WD,Normal,208500,42.023708,-93.647577
1,3,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,College Creek,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,None,None,0,9,2008,WD,Normal,223500,42.023708,-93.647577
2,14,RL,91.0,10652,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,College Creek,Norm,Norm,1Fam,1Story,7,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,Stone,306.0,Gd,TA,PConc,Gd,TA,Av,Unf,0,Unf,0,1494,1494,GasA,Ex,Y,SBrkr,0,1494,0,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,RFn,3,840,TA,TA,Y,160,33,0,0,0,0,0,None,None,0,8,2007,New,Partial,279500,42.023708,-93.647577
3,23,RL,75.0,9742,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,College Creek,Norm,Norm,1Fam,1Story,8,5,2002,2002,Hip,CompShg,VinylSd,VinylSd,BrkFace,281.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,1777,1777,GasA,Ex,Y,SBrkr,0,1795,0,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,RFn,2,534,TA,TA,Y,171,159,0,0,0,0,0,None,None,0,9,2008,WD,Normal,230000,42.023708,-93.647577
4,33,RL,85.0,11049,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,College Creek,Norm,Norm,1Fam,1Story,8,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Ex,TA,Av,Unf,0,Unf,0,1234,1234,GasA,Ex,Y,SBrkr,0,1234,0,0,2,0,3,1,Gd,7,Typ,0,None,Attchd,RFn,2,484,TA,TA,Y,0,30,0,0,0,0,0,None,None,0,1,2008,WD,Normal,179900,42.023708,-93.647577


In [13]:
# CREDENTIALS
CLIENT_ID = 'JNZADFO4ARPJWYOERXYHNDL1YGODIH0PU3ZMEEX0534A3YUW' # Foursquare ID
CLIENT_SECRET = 'UUGPYMXNWYLOCJ4IL0PTHQN5UNF3PSQEA5DTOGFDRELWE10C' # Foursquare Secret
VERSION = '20200728' # Foursquare API version

LIMIT = 100
radius = 1000 # 1 km

In [14]:
ames_venues = getNearbyVenues(nbh_df['Neighborhood'],
                              nbh_df['Latitude'],
                              nbh_df['Longitude'],
                              CLIENT_ID,
                              CLIENT_SECRET,
                              VERSION,
                              radius,
                              LIMIT)

In [15]:
ames_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bloomington Heights,42.055931,-93.630297,Bar,42.054446,-93.622896,Bar
1,Bloomington Heights,42.055931,-93.630297,Ge-Angelo's Italian Restaurant,42.054871,-93.622739,Italian Restaurant
2,Bloomington Heights,42.055931,-93.630297,Anytime Fitness,42.054720,-93.622980,Gym / Fitness Center
3,Bloomington Heights,42.055931,-93.630297,Planet Fitness,42.049300,-93.624916,Gym / Fitness Center
4,Bloomington Heights,42.055931,-93.630297,Flame & Skewer,42.049287,-93.622321,American Restaurant


In [16]:
ames_onehot = pd.get_dummies(ames_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ames_onehot['Neighborhood'] = ames_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ames_onehot.columns[-1]] + list(ames_onehot.columns[:-1])
ames_onehot = ames_onehot[fixed_columns]

ames_grouped = ames_onehot.groupby('Neighborhood').sum().reset_index()
ames_grouped.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burrito Place,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Bookstore,College Cafeteria,College Gym,College Hockey Rink,College Rec Center,College Stadium,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Donut Shop,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Juice Bar,Lighthouse,Liquor Store,Lounge,Mexican Restaurant,Mobile Phone Shop,Mongolian Restaurant,Museum,Music Venue,Nightclub,Optical Shop,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Pub,Rental Service,Resort,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Shipping Store,Shoe Store,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Supplement Shop,Taco Place,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Video Game Store,Video Store,Wine Bar,Winery
0,Bloomington Heights,0,1,2,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,1,2,0,1,1,0,0,0,0,1,2,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,1,0,0,0,0,3,1,3,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,Bluestem,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Briardale,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Brookside,0,0,2,0,0,1,0,2,2,4,0,0,0,0,0,1,0,0,0,1,2,0,1,0,1,1,4,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,4,0,2,1,5,0,0,0,0,0,0,0,2,1,0,2,0,0,3,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,1,0,0,1,0,1,1,0,3,0,0,2,0,4,0,0,0,0,2,1,1,2,0,0,0,0,1,0,0,1,1,1,0,0,1,1,1,0
4,Clear Creek,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# Change to lower case
ames_grouped.columns = ames_grouped.columns.str.lower()

# List of list of categories
ticks = [['taco', 'restaurant', 'food', 'diner', 'coffee', 'café', 'bistro', 'breakfast spot', 'burrito', 'sandwich'], 
         ['store', 'boutique', 'shop'],
         ['bus station', 'ferry'],
         ['college'],
         ['lounge', 'nightclub', 'pub', 'bar', 'wine', 'beer', 'brewery', 'bodega'],
         ['golf'], 
         ['park', 'trail', 'tree'],
         ['grocery', 'farmers market', 'convenience', 'bakery'],
         ['salon', 'pharma', 'bank', 'atm', 'service', 'agency', 'landscaping'],
         ['gym', 'ice skating', 'yoga', 'disc golf', 'basketball court', 'baseball field', 'arcade'],
         ['historic site', 'museum', 'music venue', 'rock club', 'stadium'],
         ['spa', 'bed & breakfast', 'hotel']]

# Gather columns containing buzz works
cols = []
for i in ticks:
    tic = i
    regstr = '|'.join(tic)
    cols.append(ames_grouped.columns.str.contains(regstr))

In [18]:
# Aggregate categories
cnames = ['restaurants', 'stores', 'transport', 'school', 'nightlife', 'golf', 'park', 'grocery', 'service', 'recreational', 'cultural', 'leisure']

ames_groups = pd.DataFrame(columns = cnames)

for i, j  in zip(cols, cnames):
    ames_groups.loc[:,j] = ames_grouped.loc[:,i].sum(axis = 1)

# Convert boolean to binary 
ames_groups = (ames_groups > 0) * 1

# Add neighborhood
ames_groups['Neighborhood'] = ames_grouped.neighborhood

# Merge to master df
df = df.merge(ames_groups, on = 'Neighborhood')

In [19]:
df.head()

,Id,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Latitude,Longitude,restaurants,stores,transport,school,nightlife,golf,park,grocery,service,recreational,cultural,leisure
0,1,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,College Creek,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,None,None,0,2,2008,WD,Normal,208500,42.023708,-93.647577,1,1,0,1,1,0,0,1,1,1,1,0
1,3,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,College Creek,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,None,None,0,9,2008,WD,Normal,223500,42.023708,-93.647577,1,1,0,1,1,0,0,1,1,1,1,0
2,14,RL,91.0,10652,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,College Creek,Norm,Norm,1Fam,1Story,7,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,Stone,306.0,Gd,TA,PConc,Gd,TA,Av,Unf,0,Unf,0,1494,1494,GasA,Ex,Y,SBrkr,0,1494,0,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,RFn,3,840,TA,TA,Y,160,33,0,0,0,0,0,None,None,0,8,2007,New,Partial,279500,42.023708,-93.647577,1,1,0,1,1,0,0,1,1,1,1,0
3,23,RL,75.0,9742,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,College Creek,Norm,Norm,1Fam,1Story,8,5,2002,2002,Hip,CompShg,VinylSd,VinylSd,BrkFace,281.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,1777,1777,GasA,Ex,Y,SBrkr,0,1795,0,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,RFn,2,534,TA,TA,Y,171,159,0,0,0,0,0,None,None,0,9,2008,WD,Normal,230000,42.023708,-93.647577,1,1,0,1,1,0,0,1,1,1,1,0
4,33,RL,85.0,11049,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,College Creek,Norm,Norm,1Fam,1Story,8,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Ex,TA,Av,Unf,0,Unf,0,1234,1234,GasA,Ex,Y,SBrkr,0,1234,0,0,2,0,3,1,Gd,7,Typ,0,None,Attchd,RFn,2,484,TA,TA,Y,0,30,0,0,0,0,0,None,None,0,1,2008,WD,Normal,179900,42.023708,-93.647577,1,1,0,1,1,0,0,1,1,1,1,0


In [20]:
df.to_csv('df_dataCollection.csv', index = False)